<a href="https://colab.research.google.com/github/thamizhannal/eva5/blob/master/EVA5Session4_v3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from __future__ import print_function
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [26]:
!pip install torchsummary
from torchsummary import summary


In [27]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels=8, kernel_size=3, padding=0) 
        self.bn1 = nn.BatchNorm2d(8)
        
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels=16, kernel_size=3, padding=0) #24
        self.bn2 = nn.BatchNorm2d(16)
        #self.dp1 = nn.Dropout2d(0.1)

        self.conv3 = nn.Conv2d(in_channels = 16, out_channels=20, kernel_size=3, padding=0) # 22
        self.bn3 = nn.BatchNorm2d(20)
        #self.dp2 = nn.Dropout2d(0.1)


        self.pool1 = nn.MaxPool2d(kernel_size=2) # 11
        # Apply 1x1 & reduce channel size

        self.conv4 = nn.Conv2d(in_channels = 20, out_channels=8, kernel_size=1, padding=0)
        self.bn4 = nn.BatchNorm2d(8)
        
        self.conv5 = nn.Conv2d(in_channels = 8, out_channels=16, kernel_size=3, padding=0)
        self.bn5 = nn.BatchNorm2d(16)
        #self.dp3 = nn.Dropout2d(0.1)

        self.conv6 = nn.Conv2d(in_channels = 16, out_channels=20, kernel_size=3, padding=0) # nn.Conv2d(in_channels=24, out_channels=16, kernel_size=1, padding=1)
        self.bn6 = nn.BatchNorm2d(20)
        #self.dp4 = nn.Dropout2d(0.1)

        self.pool2 = nn.AvgPool2d(kernel_size=2)
        
        # Apply 1x1 & reduce channel size
        
        #self.conv8 = nn.Conv2d(24, 24, 1, padding=0)
        #self.bn8 = nn.BatchNorm2d(24)
        
        #self.conv6 = nn.Conv2d(16, 32, 3)
        #self.bn6 = nn.BatchNorm2d(32)

        self.conv7 = nn.Conv2d(in_channels=20, out_channels=10, kernel_size=3)
        

    def forward(self, x):
        x = self.pool1(F.relu(self.bn3(self.conv3(F.relu(self.bn2(self.conv2(F.relu(self.bn1(self.conv1(x))))))))))
        #x = self.bn7(self.conv1_1x1(x))
        #x = self.pool2(F.relu(self.bn6(self.conv6(F.relu(self.bn5(self.conv5(F.relu(self.bn4(self.conv4(x))))))))))
        #x = self.pool2(F.relu(self.bn6(self.conv6(F.relu(self.bn5(self.conv5(F.relu(self.bn4(self.conv4(x))))))))))
        x = self.pool2(F.relu(self.bn6(self.conv6(F.relu(F.relu(self.bn5(self.conv5(F.relu(self.bn4(self.conv4(x)))))))))))
        # F.relu(self.bn6(self.conv6(F.relu(
        #x = F.relu(self.bn8(self.conv8(x)))
        #x = F.relu(self.bn6(self.conv6(F.relu(self.bn5(self.conv5(x))))))
        #x = F.relu(self.bn6(self.conv6(F.relu(self.bn5(self.conv5(x))))))
        #x = F.relu(self.conv7(x))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [28]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
       BatchNorm2d-2            [-1, 8, 26, 26]              16
            Conv2d-3           [-1, 16, 24, 24]           1,168
       BatchNorm2d-4           [-1, 16, 24, 24]              32
            Conv2d-5           [-1, 20, 22, 22]           2,900
       BatchNorm2d-6           [-1, 20, 22, 22]              40
         MaxPool2d-7           [-1, 20, 11, 11]               0
            Conv2d-8            [-1, 8, 11, 11]             168
       BatchNorm2d-9            [-1, 8, 11, 11]              16
           Conv2d-10             [-1, 16, 9, 9]           1,168
      BatchNorm2d-11             [-1, 16, 9, 9]              32
           Conv2d-12             [-1, 20, 7, 7]           2,900
      BatchNorm2d-13             [-1, 20, 7, 7]              40
        AvgPool2d-14             [-1, 2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [29]:

torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [30]:
from tqdm import tqdm

def train(model, device, train_loader, optimizer, epoch):
    train_loss = 0
    train_correct = 0
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        
        #train_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        #train_pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        #train_correct += train_pred.eq(target.view_as(train_pred)).sum().item()

    #train_losss /= len(train_loader.dataset)
    #print('\nTrain Set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
    #    train_loss, train_correct, len(train_loader.dataset),
    #    100. * train_correct / len(train_loader.dataset)))
       
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [31]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print("epoch:{}".format(epoch))
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/1875 [00:00<?, ?it/s]

epoch:1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0404, Accuracy: 9865/10000 (98.65%)
epoch:2


  0%|          | 0/1875 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0235, Accuracy: 9932/10000 (99.32%)
epoch:18


  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0198, Accuracy: 9937/10000 (99.37%)
epoch:19


loss=6.286335701588541e-05 batch_id=1874: 100%|██████████| 1875/1875 [00:24<00:00, 77.54it/s]



Test set: Average loss: 0.0208, Accuracy: 9934/10000 (99.34%)
